현재 파일 구조
resource/pass/ : 정상
resource/default/ : 디폴트
resource/defect/ : 이슈

알고리즘
1. 정상, 디폴트 테마 내의 xml을 기준으로 동일한 레이아웃 분류
2. 이슈 내의 xml을 분류 기준에 맞춤
3. 분류된 레이아웃 내 요소를 바탕으로 이슈 확인 - rule base
  a. 라디오 버튼 이슈 확인
  b. 아이콘 정렬 확인
  c. 전화버튼 텍스트 중앙 정렬 확인
  d. 완벽하게 동일한 아이콘 확인
  e. 가독성 확인

In [1]:
# 0. 이미지 라벨을 이용해서 1차 분류 -> xml 분류의 성능을 높이기 위한 전 작업. 
# 라벨이 같은 경우 동일한 이미지라고 가정

# 0.1 이미지 이름의 숫자는 제거함
import glob
import os
import re
import pandas as pd
import shutil

def clean(filename):
    filename = re.sub(r'\d+', '', filename)
    filename = re.sub('__','',filename)
    filename = os.path.splitext(filename)[0]
    return filename

def move_image(resource_dirs, output_dir):
    issue_dict = {}
    for resource_dir in resource_dirs:
        for theme_id in os.listdir(resource_dir):
            theme_path = os.path.join(resource_dir, theme_id)
            img_list = os.listdir(theme_path)
            print(img_list)
            if not os.path.isdir(theme_path):
                continue
            for img_file in img_list:
                
                img_path = os.path.join(theme_path, img_file)
                issue = clean(img_file)
                if issue not in issue_dict:
                    issue_dict[issue] = []
                save_name = f"{theme_id}_{img_file}"
                issue_dict[issue].append((img_path, save_name))
    print(issue_dict)

    for issue, files in issue_dict.items():
        issue_folder = os.path.join(output_dir, issue)
        os.makedirs(issue_folder, exist_ok=True)
        for src_path, save_name in files:
            dst_path = os.path.join(issue_folder, save_name)
            shutil.copy2(src_path, dst_path)

In [2]:
resource_dirs = ['./resource/pass', './resource/default']
output_dir = './output'
move_image(resource_dirs, output_dir)

['com.sec.android.app.launcher_LauncherActivity_20250513_113831.xml', 'com.samsung.android.calendar_MainActivity_20250513_115630.png', 'com.samsung.android.dialer_IntegratedSearchActivity_20250513_114057.xml', 'com.sec.android.app.launcher_LauncherActivity_20250513_111501.png', 'com.sec.android.app.launcher_LauncherActivity_20250513_111518.xml', 'com.samsung.android.dialer_DialtactsActivity_20250513_114019.xml', 'com.sec.android.gallery3d_GalleryActivity_20250513_115146.xml', 'com.sec.android.app.launcher_SearchMainSettingActivity_20250513_113840.png', 'com.samsung.android.app.routines_RoutineMainTabActivity_20250513_120408.png', 'com.samsung.android.messaging_ConversationComposer_20250513_114606.png', 'com.samsung.android.honeyboard_WritingAssistSettings_20250513_120258.xml', 'com.sec.android.app.launcher_LauncherActivity_20250513_111434.xml', 'com.sec.android.app.launcher_LauncherActivity_20250513_111501.xml', 'com.samsung.android.messaging_SearchActivity_20250513_114638.xml', 'com.s

In [3]:
## etc. 테마 내의 이슈만 뽑기

def theme_issues(theme_dir):
    theme_id = os.path.dirname(theme_dir).split('/')[-1]
    image_list = os.listdir(theme_dir)
    issue_list = set()
    for image_path in image_list:
        image_name = os.path.splitext(os.path.basename(image_path))[0]
        image_name = clean(image_name)
        issue_list.add(image_name)
    return theme_id, list(issue_list)


In [3]:
# 1. 정상, 디폴트 테마를 이슈별로 구분하였고, 폴더 내의 xml을 기준으로 동일한 레이아웃 분류
## 1.1-a xml 파일의 유사도 확인 : 문자열 유사도
'''두 시퀀스에서 최장 공통 부분 시퀀스를 찾아 유사도를 계산하는 방식. 
순서가 중요한 데이터에서의 유사도 측정에 적합하다.'''
import xml.etree.ElementTree as ET
from difflib import SequenceMatcher

def normalize_xml(path):
    xml = ET.parse(path)
    root = xml.getroot()
    return ET.tostring(root, encoding='unicode')

def text_similarity(args):
    i, j = args
    xml1 = normalize_xml(i)
    xml2 = normalize_xml(j)
    similarity = SequenceMatcher(None, xml1, xml2).ratio() * 100
    return (i, j, similarity)


In [5]:
## 1.1-b xml 파일의 유사도 확인 : zss방식
'''
트리 구조 사이의 계층적 구조에 대한 구조적 유사도 계산하는 방식.
부모-자식 구조를 가진 데이터의 구조적 유사도를 측정한다.
'''
from zss import Node, simple_distance

def xml_to_zss_node(elem):
    node = Node(elem.tag)
    for child in elem:
        node.addkid(xml_to_zss_node(child))
    return node

def similarity(xml_path1: str, xml_path2: str):
    xml1 = ET.parse(xml_path1)
    xml2 = ET.parse(xml_path2)
    node1 = xml_to_zss_node(xml1.getroot())
    node2 = xml_to_zss_node(xml2.getroot())
    max_size = max(len(list(xml1.iter())), len(list(xml2.iter())))
    distance = simple_distance(node1, node2)
    similarity = 1 - (distance / max_size)
    return similarity * 100


In [6]:
# 다른 구조를 가진 데이터를 실험 -> 낮게 나올수록 좋음
xmlpath1 = "./resource/test/test1.xml"
xmlpath2 = "./resource/test/test2.xml"
print(f"문자열 유사도: {text_similarity(xmlpath1, xmlpath2):.2f}%")
print(f"zss 유사도: {similarity(xmlpath1, xmlpath2):.2f}%")

TypeError: text_similarity() takes 1 positional argument but 2 were given

In [ ]:
# 같은 구조를 가진 데이터를 실험 -> 높게 나올수록 좋음
xmlpath1 = "./resource/test/test2.xml"
xmlpath2 = "./resource/test/test3.xml"
print(f"문자열 유사도: {text_similarity(xmlpath1, xmlpath2):.2f}%")
print(f"zss 유사도: {similarity(xmlpath1, xmlpath2):.2f}%")

## 결론 : 차이가 더 벌어지는 문자열 유사도 방식이 나은것 같음

In [4]:
# 1.2 유사도를 바탕으로한 그룹화
import glob
from tqdm import tqdm
import concurrent.futures

def group_by_similarity(xml_list, threshold: float):
    groups = []
    assigned = [False] * len(xml_list)


    for i, xml_path1 in enumerate(tqdm(xml_list, desc="그룹화 진행중...")):
        if assigned[i]:
            continue

        group = [xml_list[i]]
        assigned[i] = True

        compare_args = [(xml_path1, xml_list[j]) for j in range(i+1, len(xml_list)) if not assigned[j]]

        with concurrent.futures.ProcessPoolExecutor() as executor:
            results = list(executor.map(text_similarity, compare_args))

        for _, xml_path2, similarity in results:
            j = xml_list.index(xml_path2)
            if similarity >= threshold:
                group.append(xml_list[j])
                assigned[j] = True

        groups.append(group)

    return groups

In [5]:
import os

def calculate_group(xml_dir, threshold: float):
    xml_list = [os.path.join(xml_dir, xml) for xml in os.listdir(xml_dir) if xml.endswith('.xml')]
    print(xml_list)
    groups = group_by_similarity(xml_list, threshold)
    return groups


def all_group_by_similarity(threshold: float):
    xml_list = []
    for lable in os.listdir("./output"):
        theme_dir = os.path.join("./output", lable)
        theme_dir = glob.escape(theme_dir)
        xml_list.extend(glob.glob(f"{theme_dir}/*.xml"))
    print(xml_list)
    if len(xml_list) == 0:
        return []
    groups = group_by_similarity(xml_list, threshold)
    return groups

In [6]:
groups = calculate_group('./output/com.android.intentresolver_ChooserActivityLauncher',90)


['./output/com.android.intentresolver_ChooserActivityLauncher/[CoguL.S]Fantasy Flower Vase_Premium Video & Sound Pack_000008216867_com.android.intentresolver_ChooserActivityLauncher_20250523_104502.xml', './output/com.android.intentresolver_ChooserActivityLauncher/[KOH] Theme CE02 Yellow Duck Surfing The Waves_000008215880_com.android.intentresolver_ChooserActivityLauncher_20250516_155521.xml', './output/com.android.intentresolver_ChooserActivityLauncher/[KOH] Theme CE10 Duckling In Dreamy Sky Pond_000008216723_com.android.intentresolver_ChooserActivityLauncher_20250519_131302.xml', './output/com.android.intentresolver_ChooserActivityLauncher/[LIFEWIND] The Sun Over The Purple Sea Of Waves (Theme)_000008195917_com.android.intentresolver_ChooserActivityLauncher_20250509_165029.xml', './output/com.android.intentresolver_ChooserActivityLauncher/[CoguL.U]Black Red Rose Bat_Premium Video & Sound Pack_000008216450_com.android.intentresolver_ChooserActivityLauncher_20250523_155619.xml', './ou

그룹화 진행중...: 100%|██████████| 23/23 [00:01<00:00, 13.35it/s]


In [9]:
def move_groups(xml_groups_dir):
    groups = os.listdir(xml_groups_dir)
    for group in tqdm(groups, desc="그룹 이동 중..."):
        group_dir = os.path.join(xml_groups_dir, group)
        classified_groups = calculate_group(group_dir, 90)
        # classified_groups: [[xml1, xml2, ...], [xml3, ...], ...]
        # 그룹별로 이동
        for group_idx, classified_group in enumerate(classified_groups):
            for xml_path in classified_group:
                xml_name = os.path.basename(xml_path)
                xml_base_dir = os.path.dirname(xml_path)
                # 1. xml 이동
                new_dir = os.path.join(xml_base_dir, f"group_{group_idx}")
                os.makedirs(new_dir, exist_ok=True)
                shutil.move(xml_path, os.path.join(new_dir, xml_name))
                # 2. 동일 이름의 이미지도 이동
                image_name = os.path.splitext(xml_name)[0] + ".png"
                image_path = os.path.join(xml_base_dir, image_name)
                if os.path.exists(image_path):
                    shutil.move(image_path, os.path.join(new_dir, image_name))

In [10]:
move_groups('./output/')

그룹 이동 중...:   0%|          | 0/57 [00:00<?, ?it/s]

[]


그룹화 진행중...: 0it [00:00, ?it/s]


[]


그룹화 진행중...: 0it [00:00, ?it/s]


...

그룹 이동 중...:  96%|█████████▋| 55/57 [13:06<00:08,  4.02s/it]

['./output/com.samsung.android.messaging_SearchActivity/[LIFEWIND] Raspberry And Blueberry Pancakes (Theme)_000008195953_com.samsung.android.messaging_SearchActivity_20250509_150150.xml', './output/com.samsung.android.messaging_SearchActivity/[CDS] Surfing Ocean Beach_000008213527_com.samsung.android.messaging_SearchActivity_20250519_161353.xml', './output/com.samsung.android.messaging_SearchActivity/[LIFEWIND] Raspberry And Blueberry Pancakes (Theme)_000008195953_com.samsung.android.messaging_SearchActivity_20250509_150218.xml', './output/com.samsung.android.messaging_SearchActivity/[KOH] Theme CE06 Girl Surrounded By Violet Petals_000008215889_com.samsung.android.messaging_SearchActivity_20250520_093359.xml', './output/com.samsung.android.messaging_SearchActivity/[LIFEWIND] The Sun Over The Purple Sea Of Waves (Theme)_000008195917_com.samsung.android.messaging_SearchActivity_20250509_164559.xml', './output/com.samsung.android.messaging_SearchActivity/[KOH] Theme CE13 Lonely Iceberg F

그룹 이동 중...:  98%|█████████▊| 56/57 [13:07<00:03,  3.29s/it]

['./output/com.sec.android.app.launcher_SearchMainSettingActivity/[INV] Things Come Shapes_000008214327_com.sec.android.app.launcher_SearchMainSettingActivity_20250519_152040.xml', './output/com.sec.android.app.launcher_SearchMainSettingActivity/[HY]Summer sunset city scenery sports car_000008164057_com.sec.android.app.launcher_SearchMainSettingActivity_20250514_151835.xml', './output/com.sec.android.app.launcher_SearchMainSettingActivity/[LIFEWIND] A Baby Elephant In A Tropical Forest (Theme)_000008195867_com.sec.android.app.launcher_SearchMainSettingActivity_20250509_131630.xml', './output/com.sec.android.app.launcher_SearchMainSettingActivity/[CoguL.S]Fantasy Flower Vase_Premium Video & Sound Pack_000008216867_com.sec.android.app.launcher_SearchMainSettingActivity_20250523_102724.xml', './output/com.sec.android.app.launcher_SearchMainSettingActivity/[CDS] Surfing Ocean Beach_000008213527_com.sec.android.app.launcher_SearchMainSettingActivity_20250519_160828.xml', './output/com.sec.a

그룹 이동 중...: 100%|██████████| 57/57 [13:08<00:00, 13.83s/it]


In [43]:
# 3. 테마 이미지의 xml을 분류된 카테고리에 맞춰 분류
# 3.1 테마 이미지가 폴드인 경우에만 사용
import cv2

def _included(file_path):
    if file_path.endswith(".xml"):
        file_path = file_path.replace(".xml", ".png")
    img = cv2.imread(file_path)
    if img is None:
        print(f"이미지 파일을 찾을 수 없습니다: {file_path}")
    ratio =  (img.shape[0]/ img.shape[1])
    if ratio > 2.0:
        return False
    return True

print(_included("./resource/test/fail1.xml"))

False


In [44]:

# 3.2 폴드인 경우 가장 유사한 xml그룹 선택
def select_group(xml_path):
    if not _included(xml_path):
        return None
    
    max_similarity = 0

    groups = glob.glob("./output/*/*/", recursive=True)
    for group in tqdm(groups, desc="그룹 대표 이미지와 비교중"):
        repr_xml_path = [xml for xml in os.listdir(group) if xml.endswith(".xml")][0]
        repr_xml_path = os.path.join(group, repr_xml_path)
        _, _, similarity = text_similarity((xml_path, repr_xml_path))
        if similarity > max_similarity:
            max_similarity = similarity
            selected_group = group
    return max_similarity, selected_group


None


In [136]:
# 3.3 xml 분류를 통해 해당 그룹에서 확인해야할 이슈 확인


def find_component(image_base_dir, class_name):
    image_path = f"{image_base_dir}.png"
    xml_path = f"{image_base_dir}.xml"
    image = cv2.imread(image_path)
    if image is None:
        print(f"이미지 파일을 찾을 수 없습니다: {image_path}")
        return None

    tree = ET.parse(xml_path)
    if tree is None:
        print(f"xml 파일을 찾을 수 없습니다: {xml_path}")
        return None
    root = tree.getroot()
    components = []
    j = 0
    for elem in root.iter('node'):
        original_image = image.copy()
        if elem.attrib.get('class') == class_name:
            bounds = elem.attrib.get('bounds')
            if bounds:
                match = re.search(r'\[(\d+),(\d+)\]\[(\d+),(\d+)\]', bounds)
                if match:
                    x1, y1, x2, y2 = map(int, match.groups())
                    cv2.rectangle(original_image, (x1, y1), (x2, y2), (0, 255, 0), 2)

                    components.append((x1, y1, x2, y2))
                    j += 1
                    cv2.imwrite(f"./output/{class_name}_visualized_{j}.png", original_image)

    return components

# find_component("./resource/test/test1", "android.widget.RadioButton") 
# find_component("./resource/test/test1", "android.widget.ImageButton")
find_component("./resource/test/test1", "android.widget.TextView")
# find_component("./resource/test/test1", "android.widget.Switch")
# find_component("./resource/test/test1", "android.widget.ImageView")
# find_component("./resource/test/test1", "android.widget.FrameLayout")
# find_component("./resource/test/test1", "android.widget.LinearLayout")
# find_component("./resource/test/test1", "android.widget.RelativeLayout")


[(203, 425, 434, 502),
 (203, 502, 327, 546),
 (327, 502, 440, 546),
 (187, 1610, 376, 1654),
 (216, 1654, 346, 1695),
 (392, 1610, 581, 1654),
 (421, 1654, 551, 1695),
 (670, 1610, 712, 1697),
 (802, 1610, 991, 1697),
 (1016, 1610, 1187, 1697),
 (193, 1928, 369, 2015),
 (444, 1928, 528, 2015),
 (657, 1928, 725, 2015),
 (846, 1928, 946, 1972),
 (802, 1972, 991, 2012),
 (1063, 1928, 1140, 2015),
 (1249, 1928, 1363, 2015),
 (1438, 1928, 1584, 2015),
 (1646, 1928, 1715, 2015)]

In [2]:
# etc. 이미지 내의 컴포넌트 위치 확인

import numpy as np
def xml_visualize(image_path, xml_path):
    image_base_name = os.path.splitext(os.path.basename(image_path))[0]
    if not os.path.exists(xml_path):
        print(f"xml 파일이 존재하지 않습니다. {xml_path}")
        return 
    image = cv2.imread(image_path)
    # 이미지가 없을 수도 있으니, 크기만 참고하고 흰 배경 생성
    height, width = image.shape[:2] if image is not None else (2340, 1080)
    background = np.ones((height, width, 3), dtype=np.uint8) * 255

    tree = ET.parse(xml_path)
    root = tree.getroot()
    
    for elem in root.iter('node'):
        bounds = elem.attrib.get('bounds')
        if bounds:
            match = re.search(r'\[(\d+),(\d+)\]\[(\d+),(\d+)\]', bounds)
            if match:
                x1, y1, x2, y2 = map(int, match.groups())
                cv2.putText(background, elem.attrib.get('class'), (x1, y1), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
                cv2.rectangle(background, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(image, elem.attrib.get('class'), (x1, y1), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
                cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
    
    cv2.imwrite(f"./output/{image_base_name}_mask.png", background)
    cv2.imwrite(f"./output/{image_base_name}_visualized.png", image)

In [27]:
xml_visualize("./resource/test/test1.png", "./output/com.android.intentresolver_ChooserActivityLauncher/group_1/[CoguL.U]Black Red Rose Bat_Premium Video & Sound Pack_000008216450_com.android.intentresolver_ChooserActivityLauncher_20250523_155619.xml")

In [48]:
# etc. xml 내의 계층 구조별로 동일한 상위에 있는 컴포넌트만 따로 그림으로 그리기
import cv2
import re
import random

def get_bounds(bounds_str):
    match = re.search(r'\[(\d+),(\d+)\]\[(\d+),(\d+)\]', bounds_str)
    x1, y1, x2, y2 = map(int, match.groups())
    return x1, y1, x2, y2

def get_nodes_same_level(node, level, result):
    if len(result) <= level:
        result.append([])
    result[level].append(node)
    for child in node.findall('node'):
        get_nodes_same_level(child, level+1, result)

def draw_all_nodes(nodes, image_path, file_name='output.png'):
    img = cv2.imread(image_path)
    for node in nodes:
        bounds = node.attrib.get('bounds')
        if bounds:
            x1, y1, x2, y2 = get_bounds(bounds)
            random_color = (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))
            cv2.rectangle(img, (x1, y1), (x2, y2), random_color, 2)
            cv2.putText(img, node.attrib.get('class'), (x1, y1), cv2.FONT_HERSHEY_SIMPLEX, 0.5, random_color, 2)
    cv2.imwrite(file_name, img)    
    
def draw_components(components, image_path, file_name='output.png'):
    img = cv2.imread(image_path)
    for component in components:
        random_color = (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))
        cv2.rectangle(img, (component[0], component[1]), (component[2], component[3]), random_color, 2)
    cv2.imwrite(file_name, img)


In [81]:
# 동일한 상위의 컴포넌트간의 정렬을 확인하기

def is_contained(inner, outer):
    return outer[0] <= inner[0] and outer[1] <= inner[1] and outer[2] >= inner[2] and outer[3] >= inner[3]

def filter_components(components):
    valid_components = []
    for i, comp in enumerate(components):
        contained = False
        for j, comp_b in enumerate(components):
            if i != j and is_contained(comp, comp_b):
                contained = True
                break
        if not contained:
            valid_components.append(comp)
    return valid_components


def get_valid_components(nodes):
    components = [get_bounds(node.get('bounds')) for node in nodes]
    valid_components = []
    if len(components) == 1:
        return None
    else:
        for component in components:
            if abs(component[0]-component[2]) < 1812 * 0.5 and abs(component[1]-component[3]) < 2176 * 0.5:
                valid_components.append(component)

    valid_components = filter_components(valid_components)
    return valid_components


In [133]:
def is_centered(line):
    centers = [ ((x1 + x2) // 2) for (x1, y1, x2, y2) in line]
    gaps = [centers[i+1] - centers[i] for i in range(len(centers)-1)]
    return all(gap == gaps[0] for gap in gaps)

def is_left(line):
    lefts = [x1 for (x1, y1, x2, y2) in line]
    gaps = [lefts[i+1] - lefts[i] for i in range(len(lefts)-1)]
    return all(gap == gaps[0] for gap in gaps)

def is_right(line):
    rights = [x2 for (x1, y1, x2, y2) in line]
    gaps = [rights[i+1] - rights[i] for i in range(len(rights)-1)]
    return all(gap == gaps[0] for gap in gaps)

def get_grid(components):
    grid = []
    lines = {}
    for component in components:
        horizontal = [component[1], component[3]]
        if horizontal in grid:
            index = grid.index(horizontal)
            lines[index].append(component)
        else:
            grid.append(horizontal)
            lines[len(grid)-1] = [component]

    for i in range(len(lines)):
        line = lines[i]
        if line[i] is None or len(line) <= 2:
            return True
        elif is_centered(line) or is_left(line) or is_right(line):
            return True
        else:
            return False




In [134]:
import xml.etree.ElementTree as ET

image_path = './resource/test/fail2.png'
tree = ET.parse('./resource/test/fail2.xml')
root = tree.getroot().find('node')

levels = []
get_nodes_same_level(root, 0, levels)

all_alligned = True

for i, nodes in enumerate(levels):
    result = get_valid_components(nodes)
    if result is not None and len(result) > 0:
        grid = get_grid(result)
        if grid is False:
            all_alligned = False
            draw_components(result, image_path)
            break



0


In [3]:
# 이미지 내의 컴포넌트 안에서 visibility 이슈 확인하기
## paddle ocr 초기화

from paddleocr import PaddleOCR

paddle_ocr = PaddleOCR(
        det_model_dir='./src/weights/en_PP-OCRv3_det_infer',
        rec_model_dir='./src/weights/en_PP-OCRv3_rec_infer', 
        cls_model_dir='./src/weights/ch_ppocr_mobile_v2.0_cls_infer',
        lang='en',
        use_angle_cls=False,
        use_gpu=False,
        show_log=False)

/home/user/.pyenv/versions/themes-12/lib/python3.12/site-packages/paddle/utils/cpp_extension/extension_utils.py:711: UserWarning: No ccache found. Please be aware that recompiling all source files may be required. You can download and install ccache from: https://github.com/ccache/ccache/blob/master/doc/INSTALL.md
  warnings.warn(warning_message)
/home/user/.pyenv/versions/themes-12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
## utils
import numpy as np
def crop_component(image_path: str, component_region: tuple) -> np.ndarray:
    img = cv2.imread(image_path)
    x1, y1, x2, y2 = component_region
    component_img = img[y1:y2, x1:x2]
    return component_img

def _extract_darkest_color(component_img: np.ndarray) -> tuple:
    hsv = cv2.cvtColor(component_img, cv2.COLOR_BGR2HSV)
    dark_color = np.min(hsv, axis=0)
    return dark_color

def _extract_brightest_color(component_img: np.ndarray) -> tuple:
    hsv = cv2.cvtColor(component_img, cv2.COLOR_BGR2HSV)
    bright_color = np.max(hsv, axis=0)
    return bright_color

def _color_diff(color1: tuple, color2: tuple) -> float:
    return np.linalg.norm(np.array(color1) - np.array(color2))




In [13]:
from typing import Tuple
from sklearn.cluster import KMeans

def _get_dominant_color(pixels: np.ndarray) -> Tuple[int, int, int]:
    """
    픽셀들에서 주요 색상을 K-means로 추출
    """
    if len(pixels) == 0:
        return (0, 0, 0)
    
    # 픽셀이 너무 많으면 샘플링
    if len(pixels) > 1000:
        indices = np.random.choice(len(pixels), 1000, replace=False)
        pixels = pixels[indices]
    

    # K-means 클러스터링 (k=3)
    kmeans = KMeans(n_clusters=min(3, len(pixels)), random_state=42, n_init=10)
    kmeans.fit(pixels)
    
    # 가장 많은 픽셀을 가진 클러스터의 중심색상
    labels = kmeans.labels_
    unique_labels, counts = np.unique(labels, return_counts=True)
    dominant_cluster = unique_labels[np.argmax(counts)]
    dominant_color = kmeans.cluster_centers_[dominant_cluster]
    
    return tuple(map(int, dominant_color))
    


In [20]:
# 텍스트 상자 안의 텍스트의 주요 색상 뽑기

def extract_text_color(component_img: np.ndarray):
    result = paddle_ocr.ocr(component_img, cls=False)[0]

    # ocr 결과가 없는 경우 가장 어두운 색상과 가장 밝은 색상을 추출해 비교
    if result is None or len(result) == 0:
        dark = _extract_darkest_color(component_img)
        bright = _extract_brightest_color(component_img)
        diff = _color_diff(dark, bright)
        print(f"text_color: {dark}, {bright}, {diff}")
        return diff
    
    # ocr 결과가 있는 경우 텍스트 상자 안의 텍스트의 주요 색상 뽑기
    text_pixels = []
    
    for item in result:
        coords, (text, confidence) = item
        print(f"text: {text}, confidence: {confidence}")
        if confidence > 0.6:
            coords_array = np.array(coords, dtype=np.int32)
            text_bbox = cv2.boundingRect(coords_array)
            tx, ty, tw, th = text_bbox

            # pixel 단위로 텍스트 영역 추출
            text_region = component_img[ty:ty+th, tx:tx+tw]

            cv2.imwrite('text_region.png', text_region)
            pixels = text_region.reshape(-1, 3)
            text_pixels.extend(pixels)

    if len(text_pixels) == 0:
        dark = _extract_darkest_color(component_img)
        bright = _extract_brightest_color(component_img)
        diff = _color_diff(dark, bright)
        print(f"text_color: {dark}, {bright}, {diff}")
        return diff
    
    # 텍스트 영역의 주요 색상 추출
    text_pixels = np.array(text_pixels)
    text_color = _get_dominant_color(text_pixels)

    return text_color

In [15]:
def extract_background_color(component_img: np.ndarray):
    result = paddle_ocr.ocr(component_img, cls=False)[0]

    # ocr 결과가 없는 경우 가장 어두운 색상과 가장 밝은 색상을 추출해 비교
    if result is None or len(result) == 0:
        dark = _extract_darkest_color(component_img)
        bright = _extract_brightest_color(component_img)
        diff = _color_diff(dark, bright)
        print(f"background_color: {dark}, {bright}, {diff}")
        return diff
    
    # ocr 결과가 있는 경우 텍스트 영역을 제외한 배경 픽셀 수집
    background_mask = np.ones(component_img.shape[:2], dtype=bool)

    for item in result:
        coords, (text, confidence) = item
        if confidence > 0.6:
            coords_array = np.array(coords, dtype=np.int32)
            text_bbox = cv2.boundingRect(coords_array)
            tx, ty, tw, th = text_bbox

            background_mask[ty:ty+th, tx:tx+tw] = False

    background_pixels = component_img[background_mask]

    if len(background_pixels) == 0:
        dark = _extract_darkest_color(component_img)
        bright = _extract_brightest_color(component_img)
        diff = _color_diff(dark, bright)
        return diff
    
    background_color = _get_dominant_color(background_pixels)
    return background_color

In [21]:
import os
import cv2
import xml.etree.ElementTree as ET
import re

image_path = './resource/test/test1.png'
xml_path = './resource/test/test1.xml'
xml_visualize(image_path, xml_path)

img = cv2.imread(image_path)
x1, y1, x2, y2 = (203, 502, 327, 546)


component_img = img[y1:y2, x1:x2]
cv2.imwrite('component_img.png', component_img)

text_color = extract_text_color(component_img)
background_color = extract_background_color(component_img)

print(f"text_color: {text_color}, background_color: {background_color}")



text: 7.80MB, confidence: 0.9970420002937317
text_color: (253, 250, 250), background_color: (255, 252, 252)


In [27]:
def hsv_based_extraction(image, num_colors=3):
    """HSV 색공간 기반 색상 추출"""
    if len(image.shape) == 3:
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    else:
        image_rgb = image
        
    # HSV로 변환
    image_hsv = cv2.cvtColor(image_rgb, cv2.COLOR_RGB2HSV)
    
    # H(색조) 채널에서 주요 색조 찾기
    h_channel = image_hsv[:, :, 0]
    hist_h = cv2.calcHist([h_channel], [0], None, [180], [0, 180])
    
    # 주요 색조 피크 찾기
    peaks = []
    for i in range(1, len(hist_h) - 1):
        if hist_h[i] > hist_h[i-1] and hist_h[i] > hist_h[i+1] and hist_h[i] > np.max(hist_h) * 0.1:
            peaks.append(i)
    
    # 가장 높은 피크들 선택
    peaks = sorted(peaks, key=lambda x: hist_h[x], reverse=True)[:num_colors]
    
    # 각 주요 색조에 대해 대표 색상 찾기
    colors = []
    for peak_h in peaks:
        # 해당 색조 범위의 픽셀들 찾기
        mask = np.abs(h_channel - peak_h) < 10
        if np.any(mask):
            masked_pixels = image_rgb[mask]
            if len(masked_pixels) > 0:
                avg_color = np.mean(masked_pixels, axis=0).astype(int)
                colors.append(avg_color.tolist())
    
    return colors  


In [37]:


def lab_based_extraction(image, num_colors=3):
    """LAB 색공간 기반 색상 추출 (명도가 비슷한 이미지용)"""
    if len(image.shape) == 3:
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    else:
        image_rgb = image
    
    # LAB 색공간으로 변환
    image_lab = cv2.cvtColor(image_rgb, cv2.COLOR_RGB2LAB)
    
    # A, B 채널 사용 (색상 정보)
    a_channel = image_lab[:, :, 1]
    b_channel = image_lab[:, :, 2]
    
    # 2D 히스토그램 (A-B 평면)
    hist_ab = cv2.calcHist([a_channel, b_channel], [0, 1], None, [32, 32], [0, 256, 0, 256])
    
    # 피크 찾기
    peaks = []
    threshold = np.max(hist_ab) * 0.05
    
    for i in range(1, hist_ab.shape[0] - 1):
        for j in range(1, hist_ab.shape[1] - 1):
            if (hist_ab[i, j] > threshold and 
                hist_ab[i, j] > hist_ab[i-1, j] and hist_ab[i, j] > hist_ab[i+1, j] and
                hist_ab[i, j] > hist_ab[i, j-1] and hist_ab[i, j] > hist_ab[i, j+1]):
                peaks.append((i, j, hist_ab[i, j]))
    
    peaks = sorted(peaks, key=lambda x: x[2], reverse=True)[:num_colors]
    
    colors = []
    for a_bin, b_bin, _ in peaks:
        a_center = a_bin * 8
        b_center = b_bin * 8
        
        a_mask = np.abs(a_channel - a_center) < 12
        b_mask = np.abs(b_channel - b_center) < 12
        region_mask = a_mask & b_mask
        
        if np.any(region_mask):
            region_pixels = image_rgb[region_mask]
            avg_color = np.mean(region_pixels, axis=0).astype(int)
            colors.append(avg_color.tolist())
    
    # K-means로 보완
    if len(colors) < num_colors:
        pixels_reshaped = image_rgb.reshape(-1, 3)
        kmeans = KMeans(n_clusters=num_colors, random_state=42, n_init=10)
        kmeans.fit(pixels_reshaped)
        
        for center in kmeans.cluster_centers_:
            if len(colors) < num_colors:
                colors.append(center.astype(int).tolist())
    
    return colors[:num_colors]

def adaptive_color_extraction(image, num_colors=3):
    """적응적 색상 추출 - 이미지 특성에 따라 방법 선택"""
    if len(image.shape) == 3:
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    else:
        image_rgb = image
    
    # 이미지 특성 분석
    image_hsv = cv2.cvtColor(image_rgb, cv2.COLOR_RGB2HSV)
    h, s, v = cv2.split(image_hsv)
    
    # 채도와 명도의 분산 계산
    saturation_var = np.var(s)
    value_var = np.var(v)
    
    # 고채도 픽셀 비율
    high_sat_ratio = np.sum(s > 50) / s.size
    
    print(f"채도 분산: {saturation_var:.2f}, 명도 분산: {value_var:.2f}, 고채도 비율: {high_sat_ratio:.2f}")
    
    if high_sat_ratio < 0.3 or saturation_var < 500:
        # 저채도 이미지 - LAB 색공간 사용
        print("저채도 이미지 감지 - LAB 색공간 사용")
        return lab_based_extraction(image, num_colors)
    else:
        # 일반 이미지 - 개선된 HSV 사용
        print("일반 이미지 - 개선된 HSV 사용")
        return hsv_based_extraction(image, num_colors)

채도 분산: 278.15, 명도 분산: 60.43, 고채도 비율: 0.35
저채도 이미지 감지 - LAB 색공간 사용


[[76, 67, 79], [68, 60, 73], [84, 75, 90]]

In [33]:
# 추출된 색상의 대비 정도 확인
def normalize_color(c):
    c = c / 255.0
    if c < 0.03928:
        return c / 12.92
    else:
        return ((c + 0.055) / 1.055) ** 2.4

def get_luminance(color):
    r, g, b = color
    r_luminance = normalize_color(r)
    g_luminance = normalize_color(g)
    b_luminance = normalize_color(b)
    luminance = 0.2126 * r_luminance + 0.7152 * g_luminance + 0.0722 * b_luminance
    return luminance

def calculate_contrast(colors):
    for i in range(len(colors)):
        for j in range(i+1, len(colors)):
            luminance1 = get_luminance(colors[i])
            luminance2 = get_luminance(colors[j])

            lighter = max(luminance1, luminance2)
            darker = min(luminance1, luminance2)

            contrast_ratio = (lighter + 0.05) / (darker + 0.05)

            if contrast_ratio > 3.0:
                return True
    return False

In [42]:
image = cv2.imread('./fail2.png')
colors = adaptive_color_extraction(image)
print(calculate_contrast(colors))

채도 분산: 1383.81, 명도 분산: 225.81, 고채도 비율: 0.69
일반 이미지 - 개선된 HSV 사용
False
